<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/thirdVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

Importowanie pakietów

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import csv

Ściezki plików

In [3]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
labels['PD'] = labels['PD'].replace(2, 1)
path='/content/drive/MyDrive/dyplom/EEG'

Preprocesing danych

In [ ]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
        print(filename)
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            raw.info.set_montage('standard_1020')
            raw.info['bads'].append('Oz')
            annotations = raw.annotations
            for i in range(len(annotations)):
              if annotations.description[i] == "break":
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
            raw.set_annotations(annotations)
            raw.filter(1,40, fir_design='firwin')
            events = mne.make_fixed_length_events(raw, duration=1.0)
            reject = dict(
              eeg=82e-6,
              )
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1,preload=True,reject_by_annotation='BAD',reject=reject)
            epochs_array.append([epochs,label])
            # try:
            #   epochs = ar.fit_transform(epochs)
            #   epochs_array.append([epochs,label])
            # except Exception as e:
            #   print("Wystąpił wyjątek podczas przetwarzania danych:", e)
            del epochs
            del raw
            del annotations
    return epochs_array


In [4]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
      print(filename)
      if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf(filepath, preload=True,verbose=False)
        raw.info.set_montage('standard_1020')
        raw.info['bads'].append('Oz')
        annotations = raw.annotations
        for i in range(len(annotations)):
            if annotations.description[i] == "break":
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
        raw.set_annotations(annotations)
        raw.filter(1, 40, fir_design='firwin')
        events = mne.make_fixed_length_events(raw, duration=1.0)
        unique_events, unique_indices = np.unique(events[:, 0], return_index=True)
        events = events[unique_indices]
        tmin = 0
        tmax = 2
        baseline = (0, 2)
        reject = dict(
              eeg=80e-6,
              )
        epochs = mne.Epochs(raw, events, event_id=None, tmin=tmin, tmax=tmax, baseline=baseline, preload=True,reject_by_annotation='BAD',reject=reject,verbose=False)
        epochs_array.append([epochs, label])
        del raw
        del annotations
    return epochs_array


PSD

In [14]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        if epochs is not None and len(epochs) > 0:
            epochs_data = epochs.get_data()
            epo_spectrum = mne.time_frequency.psd_array_multitaper(epochs_data, 512, fmin=1, fmax=8, bandwidth=4, adaptive=True, normalization='length', output='power', n_jobs=-1, max_iter=100, verbose=False)
            psds_epoch, freqs = epo_spectrum
            combined_psds = []
            for p in psds_epoch:
                p = 10 * np.log10(p)
                psds_mean = p.mean(0)
                psds_std = p.std(0)
                combined_psds.append(psds_mean)
            combined_psds = np.array(combined_psds)
            data.append([combined_psds, label])

            del psds_epoch
            del freqs
            del epo_spectrum
    return data


In [ ]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [15]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

Tworzenie zestawów

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['psds'], df['label'], test_size=0.1, random_state=42)
X_train_2d = np.array([x[0] for x in X_train[2:]]).reshape(len(X_train)-2, -1)
X_test_2d = np.array([x[0] for x in X_test[2:]]).reshape(len(X_test)-2, -1)

y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()

In [38]:
print(len(X_train_2d))
print(len(y_train))
for p in X_train_2d:
  print((p))

92
94
[-82.67334904 -82.32855763 -82.49236944 -83.15148502 -87.18696667
 -88.66139651 -89.36519802 -89.40172581 -90.60667802 -88.88077165
 -88.26885714 -88.13342827 -89.22280064 -88.97997656]
[-84.5055378  -84.11921501 -84.71928664 -85.8959336  -88.69942093
 -89.40146642 -89.63347987 -90.53866701 -90.62526214 -90.27292038
 -89.99230411 -89.66167925 -89.54932254 -88.09728092]
[-86.16297556 -86.43387907 -88.70754525 -90.32607424 -91.00867653
 -92.02796835 -91.64352324 -91.48352459 -91.67392917 -91.66047101
 -91.78777633 -92.29448436 -92.5509251  -92.95055821]
[-86.75130775 -86.62197672 -87.35295869 -89.34394133 -90.64441406
 -91.56201069 -92.78441    -93.17347958 -93.26627463 -94.47309522
 -94.18962317 -94.10033455 -94.18430294 -94.60991344]
[-86.40532513 -86.28666926 -86.90624528 -88.94701484 -87.55824341
 -86.47025153 -85.61250787 -85.16478981 -85.27671602 -85.47091798
 -85.14295032 -85.35061766 -86.91158333 -88.09547539]
[-82.11511239 -82.08035016 -83.11414875 -83.81001232 -84.9075873

In [23]:
from mne.decoding import Vectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Tworzenie i trenowanie modelu SVM
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), SVC())
clf_svm_pip.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
predictions_svm = clf_svm_pip.predict(X_test_2d)

# Raport klasyfikacji
report_svm = classification_report(y_test, predictions_svm, target_names=['0', '1'])
print('SVM Classification Report:\n{}'.format(report_svm))

# Dokładność modelu SVM
acc_svm = accuracy_score(y_test, predictions_svm)
print("Accuracy of SVM model: {}".format(acc_svm))

# Obliczanie precyzji, czułości i miary F1
precision_svm, recall_svm, fscore_svm, support_svm = precision_recall_fscore_support(y_test, predictions_svm, average='macro')
print('Precision: {0}, Recall: {1}, F1-score: {2}'.format(precision_svm, recall_svm, fscore_svm))

# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf_svm_pip, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.86      0.86      0.86         7

    accuracy                           0.82        11
   macro avg       0.80      0.80      0.80        11
weighted avg       0.82      0.82      0.82        11

Accuracy of SVM model: 0.8181818181818182
Precision: 0.8035714285714286, Recall: 0.8035714285714286, F1-score: 0.8035714285714286
Cross-Validation Results (5-fold)
Mean Accuracy: 0.5426900584795321
Standard Deviation: 0.11245113770032202


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import Vectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import cross_val_score
# Definiowanie modelu
clf = make_pipeline(
    StandardScaler(),
    Vectorizer(),
    RandomForestClassifier()
)
# Trenowanie modelu na wszystkich danych treningowych
clf.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)

# Raport klasyfikacji
report = classification_report(y_test, y_pred, target_names=['0', '1'])
print('Random Forest Classification Report:\n{}'.format(report))

# Dokładność modelu
acc = accuracy_score(y_test, y_pred)
print("Accuracy of model: {}".format(acc))

# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.86      0.92         7

    accuracy                           0.91        11
   macro avg       0.90      0.93      0.91        11
weighted avg       0.93      0.91      0.91        11

Accuracy of model: 0.9090909090909091
Cross-Validation Results (5-fold)
Mean Accuracy: 0.575438596491228
Standard Deviation: 0.08406420035586624


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import Vectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
clf = make_pipeline(
StandardScaler(),
Vectorizer(),
LogisticRegression() # Używamy regresji logistycznej z domyślnymi parametrami
)
clf.fit(X_train_2d, y_train)

#Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))
# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

Accuracy score: 0.8181818181818182
Cross-Validation Results (5-fold)
Mean Accuracy: 0.5736842105263158
Standard Deviation: 0.13562209186026447


In [ ]:
from joblib import dump, load
dump(clf, 'oficjalnyRandomForest.joblib')

['oficjalnyRandomForest.joblib']